# Retinal Vessel

The retinal vessel segmentation is a binary image segmentation. Currently there are 4 well used dataset are available for this task. [HERE](https://www.kaggle.com/ipythonx/retinal-vessel-segmentation) is the dataset all together. 

```
CHASE DB1
DRIVE
HRF
STARE
```

In this starter notebook, we will explore all of them but additionally we will run a simple segmentation model on **DRIVE** dataset only. Rest of the datasets are pretty same format and should be easy to run.

In [ ]:
!pip install -U -q segmentation-models --user

from PIL import Image
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["SM_FRAMEWORK"] = "tf.keras"

import segmentation_models as sm
import matplotlib.pyplot as plt 
import tensorflow_io as tfio
import tensorflow as tf
from sklearn.model_selection import train_test_split


root = '../input/retinal-vessel-segmentation'
exts = ('jpg', 'JPG', 'png', 'PNG', 'tif', 'gif', 'ppm')

# DRIVE

The dataset comes with pair of input retina image and target mask. Among all retina image, we will only use this dataset for a quick baseline. However, rest of the dataset can be replaces easily on this pipeline.


## Preprocessing Images

In [ ]:
def restrict_normalized(imgs):
    imgs_normalized = np.empty(imgs.shape)
    imgs_std = np.std(imgs)
    imgs_mean = np.mean(imgs)
    imgs_normalized = (imgs-imgs_mean)/imgs_std
    imgs_normalized = ((imgs_normalized - np.min(imgs_normalized)) / (np.max(imgs_normalized)-np.min(imgs_normalized)))*255
    return imgs_normalized

# CLAHE (Contrast Limited Adaptive Histogram Equalization)
#adaptive histogram equalization is used. In this, image is divided into small blocks called "tiles" (tileSize is 8x8 by default in OpenCV). Then each of these blocks are histogram equalized as usual. So in a small area, histogram would confine to a small region (unless there is noise). If noise is there, it will be amplified. To avoid this, contrast limiting is applied. If any histogram bin is above the specified contrast limit (by default 40 in OpenCV), those pixels are clipped and distributed uniformly to other bins before applying histogram equalization. After equalization, to remove artifacts in tile borders, bilinear interpolation is applied
def clahe_equalized(imgs):
  clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(16,16))
  imgs_equalized = np.empty(imgs.shape)
  imgs_equalized = clahe.apply(np.array(imgs, dtype = np.uint8))
  return imgs_equalized

def normalized(imgs):
  imgs_normalized =np.empty(imgs.shape)
  imgs_normalized =cv2.equalizeHist(imgs)
  return imgs_normalized

def adjust_gamma(imgs, gamma=1.0):
  invGamma = 1.0 / gamma
  table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
  # apply gamma correction using the lookup table
  new_imgs = np.empty(imgs.shape)
  new_imgs = cv2.LUT(np.array(imgs, dtype = np.uint8), table)
  return new_imgs

def expand(image):
  arr = np.zeros(image.shape + (3,))
  arr[:,:,0] = image[:,:]
  arr[:,:,1] = image[:,:]
  arr[:,:,2] = image[:,:]
  return arr

def preprocess(image):
  image=restrict_normalized(image)
  image=clahe_equalized(image)
  image=adjust_gamma(image,1.2)
  image=image/255.0
  image=expand(image)
  return image

In [ ]:
import cv2
from PIL import Image

processed_path = './Processed_images_Drive'

os.mkdir(path=processed_path)
directory = '../input/retinal-vessel-segmentation/DRIVE/training/images/'
for fname in os.listdir(directory):
    image = plt.imread(directory + fname)
    image = image[:,:,1]
    processed_img = preprocess(image)
    print(processed_img.shape)
    processed_img = Image.fromarray((processed_img * 255).astype(np.uint8))
    processed_img.save(processed_path + '/' + fname)

In [ ]:
input_data = os.path.join('./', 'Processed_images_Drive')
images = sorted(
    [
        os.path.join(input_data, fname)
        for fname in os.listdir('./Processed_images_Drive')
        if fname.endswith(exts) and not fname.startswith(".")
    ]
)


target_data = os.path.join(root, 'DRIVE/training/1st_manual')
masks = sorted(
    [
        os.path.join(target_data, fname)
        for fname in os.listdir(target_data)
        if fname.endswith(exts) and not fname.startswith(".")
    ]
)

print("Number of samples:", len(images), len(masks))
for input_path, target_path in zip(images[:10], masks[:10]):
    print(input_path[-31:], "|", target_path[-34:])

In [ ]:
imagesss = cv2.imread('./Processed_images_Drive/22_training.tif')
print(imagesss.shape)
plt.imshow(imagesss)

In [ ]:
IMAGE_SIZE = 512
BATCH_SIZE = 12

def read_files(image_path, mask=False):
    image = tf.io.read_file(image_path)
    if mask:
        image = tf.io.decode_gif(image) # out: (1, h, w, 3)
        image = tf.squeeze(image) # out: (h, w, 3)
        image = tf.image.rgb_to_grayscale(image) # out: (h, w, 1)
        image = tf.divide(image, 128)
        image.set_shape([None, None, 1])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
        image = tf.cast(image, tf.int32)
    else:
        image = tfio.experimental.image.decode_tiff(image) # out: (h, w, 4)
        image = image[:,:,:3] # out: (h, w, 3)
        image.set_shape([None, None, 3])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
        image = image / 255.
    return image

def load_data(image_list, mask_list):
    image = read_files(image_list)
    mask  = read_files(mask_list, mask=True)
    return image, mask

def data_generator(image_list, mask_list):
    dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset1 = dataset.take(15)
    dataset2 = dataset.skip(15)
    dataset1 = dataset1.batch(BATCH_SIZE, drop_remainder=False)
    dataset2 = dataset2.batch(2, drop_remainder=False)
    return dataset1, dataset2

train_dataset, valid_dataset = data_generator(images, masks)
train_dataset, valid_dataset

In [ ]:
def visualize(**images):
    """PLot images in one row."""
    n = len(images) 
    plt.figure(figsize=(20, 20))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image, cmap='gray')
    plt.show()

In [ ]:
image, mask = next(iter(train_dataset.take(1))) 
print(image.shape, mask.shape)

for (img, msk) in zip(image[:5], mask[:5]):
    print(mask.numpy().min(), mask.numpy().max())
    print(np.unique(mask.numpy()))
    visualize(
        image=img.numpy(),
        gt_mask=msk.numpy(),
    )

# Model

In [ ]:
from tensorflow import keras 

# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()
BACKBONE   = 'vgg19'
n_classes  = 1 
activation = 'sigmoid' 
model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)
model.summary(line_length=110)

# Callback : Monitoring Training Progress

In [ ]:
class DisplayCallback(keras.callbacks.Callback):
    def __init__(self, dataset, epoch_interval=5):
        self.dataset = dataset
        self.epoch_interval = epoch_interval
    
    def display(self, display_list, extra_title=''):
        plt.figure(figsize=(15, 15))
        title = ['Input Image', 'True Mask', 'Predicted Mask']

        if len(display_list) > len(title):
            title.append(extra_title)

        for i in range(len(display_list)):
            plt.subplot(1, len(display_list), i+1)
            plt.title(title[i])
            plt.imshow(display_list[i], cmap='gray')
            plt.axis('off')
        plt.show()
        
    def create_mask(self, pred_mask):
        pred_mask = (pred_mask > 0.5).astype("int32")
        return pred_mask[0]
    
    def show_predictions(self, dataset, num=1):
        for image, mask in dataset.take(num):
            pred_mask = model.predict(image)
            self.display([image[0], mask[0], self.create_mask(pred_mask)])
        
    def on_epoch_end(self, epoch, logs=None):
        if epoch and epoch % self.epoch_interval == 0:
            self.show_predictions(self.dataset)
            print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

# Compile and Fit

In [ ]:
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau

# define optomizer
optim = keras.optimizers.Adam(0.0003)
bce   = keras.losses.BinaryCrossentropy()
metrics = ["accuracy"]

# compile keras model with defined optimozer, loss and metrics
model.compile(optim, bce, metrics)

filepath = './unet_backbone_vgg19.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
rlrop = ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience=5, factor=0.6, min_lr=1e-7, verbose=1)

history = model.fit(
    train_dataset,
    epochs=400,
    callbacks=[DisplayCallback(train_dataset), checkpoint, rlrop],
    validation_data=valid_dataset
)

In [ ]:
# plot the loss
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.legend()
plt.show()
#plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.legend()
plt.show()
#plt.savefig('AccVal_acc')

In [ ]:
tf.keras.models.save_model(model, './unet_backbone2_vgg19.h5')

## Evaluating Functions

In [ ]:
def my_dice(target, prediction):
 intersection = np.logical_and(target, prediction)
 union = np.logical_or(target, prediction)
 dice = (2*np.sum(intersection))/(np.sum(union)+np.sum(intersection))
 return dice

In [ ]:
def jaccard(target, prediction):
    intersection = np.logical_and(target, prediction)
    union = np.logical_or(target, prediction)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

In [ ]:
def quick_plot(img, msk):
    plt.figure(figsize=(10, 10))   
    plt.subplot(1, 2, 1)
    plt.axis('off')
    plt.imshow(img)
    plt.title('image')

    plt.subplot(1, 2, 2)
    plt.axis('off')
    plt.imshow(msk)
    plt.title('mask')
    plt.show()


In [ ]:
from tensorflow.keras.models import load_model

model = load_model('./unet_backbone_vgg19.h5') 

## **Evaluating on DRIVE**

In [ ]:
test_masks = np.concatenate([y for x, y in valid_dataset], axis=0)
masks = test_masks.ravel()

In [ ]:
test_imgs = np.concatenate([x for x, y in valid_dataset], axis=0)
predictions = model.predict(test_imgs)
predictions.shape

In [ ]:
y_pred = predictions.ravel()
y_pred = (y_pred > 0.5).astype('int32')
y_pred

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(masks, y_pred))

In [ ]:
print('The Jaccard Score is: ', jaccard(y_pred, masks))

In [ ]:
print('The Dice Score is: ', my_dice(y_pred, masks))

## Sample Prediction

In [ ]:
image = test_imgs[4]
mask = test_masks[4]
image = np.expand_dims(image, axis=0)
pred_mask = model.predict(image)
pred_mask = (pred_mask > 0.5).astype('int32')
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.title('Original Mask')
plt.imshow(mask, cmap='gray')
plt.axis('off')
plt.subplot(122)
plt.title('Predicted Mask')
plt.imshow(pred_mask[0], cmap='gray')
plt.axis('off')

# HRF


The **HRF** dataset comes with retina image and corresponding vessel target and retina mask itself. See the code below.

In [ ]:
hrf_dir = '../input/retinal-vessel-segmentation/HRF'

input_data = os.path.join(hrf_dir, 'images')
images = sorted(
    [
        os.path.join(input_data, fname)
        for fname in os.listdir(input_data)
        if fname.endswith(exts) and not fname.startswith(".")
    ]
)


target_data = os.path.join(hrf_dir, 'mask')
masks1 = sorted(
    [
        os.path.join(target_data, fname)
        for fname in os.listdir(target_data)
        if fname.endswith(exts) and not fname.startswith(".")
    ]
)

target_data = os.path.join(hrf_dir, 'manual1')
masks2 = sorted(
    [
        os.path.join(target_data, fname)
        for fname in os.listdir(target_data)
        if fname.endswith(exts) and not fname.startswith(".")
    ]
)

len(images), len(masks1), len(masks2)

In [ ]:
def quick_plot(img, msk1, msk2):
    plt.figure(figsize=(25, 25))   
    plt.subplot(1, 3, 1)
    plt.axis('off')
    plt.imshow(img)
    plt.title('image')

    plt.subplot(1, 3, 2)
    plt.axis('off')
    plt.imshow(msk1)
    plt.title('mask type 1')

    plt.subplot(1, 3, 3)
    plt.axis('off')
    plt.imshow(msk2)
    plt.title('mask type 2')
    plt.show() 


for img, msk1, msk2 in zip(images, masks1, masks2):
    print(img[-9:],' | ', msk1[-14:], ' | ', msk2[-9:])
    
    # read files 
    read_img  = Image.open(img)
    read_msk1 = Image.open(msk1)
    read_msk2 = Image.open(msk2)
    
    # plot them 
    print(read_img.size, read_msk1.size, read_msk2.size)
    quick_plot(read_img, read_msk1, read_msk2)

## Preprocess

In [ ]:
import cv2
from PIL import Image

processed_path = './Processed_images_HRF'

os.mkdir(path=processed_path)
directory = '../input/retinal-vessel-segmentation/HRF/images/'
for fname in os.listdir(directory):
    image = plt.imread(directory + fname)
    image = image[:,:,1]
    processed_img = preprocess(image)
    processed_img = Image.fromarray((processed_img * 255).astype(np.uint8))
    processed_img.save(processed_path + '/' + fname)

In [ ]:
input_data = os.path.join('./', 'Processed_images_HRF')
images = sorted(
    [
        os.path.join(input_data, fname)
        for fname in os.listdir('./Processed_images_HRF')
        if fname.endswith(exts) and not fname.startswith(".")
    ]
)

In [ ]:
IMAGE_SIZE = 512
BATCH_SIZE = 1

def read_files(image_path):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_image(image) # out: (h, w, 4)
    image = image[:,:,:3] # out: (h, w, 3)
    image.set_shape([None, None, 3])
    image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
    image = image / 255.
    return image

def load_data(image_list):
    image = read_files(image_list)
    return image

def data_generator(image_list):
    dataset = tf.data.Dataset.from_tensor_slices((image_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=False)
    return dataset

test_dataset = data_generator(images)
test_dataset

In [ ]:
IMAGE_SIZE = 512
BATCH_SIZE = 1

def read_files(mask_path):
    mask = tf.io.read_file(mask_path)
    mask = tfio.experimental.image.decode_tiff(mask) # out: (1, h, w, 3)
    mask = mask[:,:,:3] # out: (h, w, 3)
    mask = tf.image.rgb_to_grayscale(mask) # out: (h, w, 1)
    mask = tf.divide(mask, 128)
    mask.set_shape([None, None, 1])
    mask = tf.image.resize(images=mask, size=[IMAGE_SIZE, IMAGE_SIZE])
    mask = tf.cast(mask, tf.int32)
    return mask

def load_data(mask_list):
    mask = read_files(mask_list)
    return mask

def data_generator(mask_list):
    dataset = tf.data.Dataset.from_tensor_slices((mask_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=False)
    return dataset

mask_dataset = data_generator(masks2)
mask_dataset

In [ ]:
predictions = model.predict(test_dataset)
predictions.shape

In [ ]:
y_pred = predictions.ravel()
y_pred = (y_pred > 0.5).astype('int32')
y_pred

In [ ]:
masks = mask_dataset.unbatch()
masks = list(masks.as_numpy_iterator())
test_masks = np.array(masks)
masks = test_masks.ravel()
masks

In [ ]:
jaccard_score = jaccard(y_pred, masks)
dice = my_dice(y_pred, masks)

print('The Jaccard Score is: ', jaccard_score)
print('The Dice Score is: ', dice)
print(classification_report(masks, y_pred))

## Sample Prediction

In [ ]:
test_imgs = np.concatenate([x for x in test_dataset], axis=0)

In [ ]:
image = test_imgs[21]
mask = test_masks[21]
image = np.expand_dims(image, axis=0)
pred_mask = model.predict(image)
pred_mask = (pred_mask > 0.5).astype('int32')
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.title('Original Mask')
plt.imshow(mask, cmap='gray')
plt.axis('off')
plt.subplot(122)
plt.title('Predicted Mask')
plt.imshow(pred_mask[0], cmap='gray')
plt.axis('off')

# CHASE DB1

This data set comes with both retina image and corresponding target mask. Note that, for each retina image, there are two mask. Below, in data loading time, we only pick one mask agains the retina image. 

In [ ]:
hrf_dir = '../input/retinal-vessel-segmentation/CHASE_DB1'


input_data = os.path.join(hrf_dir, 'Images')
images = sorted(
    [
        os.path.join(input_data, fname)
        for fname in os.listdir(input_data)
        if fname.endswith(exts) and not fname.startswith(".")
    ]
)


target_data = os.path.join(hrf_dir, 'Masks')
masks = sorted(
    [
        os.path.join(target_data, fname)
        for fname in os.listdir(target_data)
        if fname.endswith('_2ndHO.png') and not fname.startswith(".") # _2ndHO.png, _1stHO.png
    ]
)


len(images), len(masks)

## Preprocess

In [ ]:
import cv2
from PIL import Image

processed_path = './Processed_images_ChaseDB1'

os.mkdir(path=processed_path)
directory = '../input/retinal-vessel-segmentation/CHASE_DB1/Images/'
for fname in os.listdir(directory):
    image = plt.imread(directory + fname)
    image = image[:,:,1]
    processed_img = preprocess(image)
    processed_img = Image.fromarray((processed_img * 255).astype(np.uint8))
    processed_img.save(processed_path + '/' + fname)

In [ ]:
input_data = os.path.join('./', 'Processed_images_ChaseDB1')
images = sorted(
    [
        os.path.join(input_data, fname)
        for fname in os.listdir('./Processed_images_ChaseDB1')
        if fname.endswith(exts) and not fname.startswith(".")
    ]
)

In [ ]:
IMAGE_SIZE = 512
BATCH_SIZE = 1

def read_files(image_path):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_image(image) # out: (h, w, 4)
    image = image[:,:,:3] # out: (h, w, 3)
    image.set_shape([None, None, 3])
    image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
    image = image / 255.
    return image

def load_data(image_list):
    image = read_files(image_list)
    return image

def data_generator(image_list):
    dataset = tf.data.Dataset.from_tensor_slices((image_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=False)
    return dataset

test_dataset = data_generator(images)
test_dataset

In [ ]:
IMAGE_SIZE = 512
BATCH_SIZE = 1

def read_files(mask_path):
    mask = tf.io.read_file(mask_path)
    mask = tf.io.decode_gif(mask) # out: (1, h, w, 3)
    mask = tf.squeeze(mask) # out: (h, w, 3)
    mask = tf.image.rgb_to_grayscale(mask) # out: (h, w, 1)
    mask = tf.divide(mask, 128)
    mask.set_shape([None, None, 1])
    mask = tf.image.resize(images=mask, size=[IMAGE_SIZE, IMAGE_SIZE])
    mask = tf.cast(mask, tf.int32)
    return mask

def load_data(mask_list):
    mask = read_files(mask_list)
    return mask

def data_generator(mask_list):
    dataset = tf.data.Dataset.from_tensor_slices((mask_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=False)
    return dataset

mask_dataset = data_generator(masks)
mask_dataset

In [ ]:
predictions = model.predict(test_dataset)
predictions.shape

In [ ]:
masks = mask_dataset.unbatch()
masks = list(masks.as_numpy_iterator())
test_masks = np.array(masks)
masks = test_masks.ravel()
masks

In [ ]:
y_pred = predictions.ravel()
y_pred = (y_pred > 0.5).astype('int32')
y_pred

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(masks, y_pred))

In [ ]:
jaccard_score = jaccard(y_pred, masks)
print('The Jaccard Score is: ', jaccard_score)

In [ ]:
dice = my_dice(y_pred, masks)
# quick_plot(mask, pred_mask)
    
print('The Dice Score is: ', dice)

In [ ]:
test_imgs = np.concatenate([x for x in test_dataset], axis=0)

In [ ]:
image = test_imgs[20]
mask = test_masks[20]
image = np.expand_dims(image, axis=0)
pred_mask = model.predict(image)
pred_mask = (pred_mask > 0.5).astype('int32')
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.title('Original Mask')
plt.imshow(mask, cmap='gray')
plt.axis('off')
plt.subplot(122)
plt.title('Predicted Mask')
plt.imshow(pred_mask[0], cmap='gray')
plt.axis('off')

# Our High Quality Image Dataset

In [ ]:
IMAGE_SIZE = 512

org_image = plt.imread('/kaggle/input/neu-2dsegmentation/segmentation/test/image/1.jpg')
image = org_image[:,:,1]
processed_img = preprocess(image)
# processed_img = Image.fromarray((processed_img * 255).astype(np.uint8))
processed_img = (processed_img * 255).astype(np.uint8)
image = tf.convert_to_tensor(processed_img)
image.set_shape([None, None, 3])
image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
image = image / 255.
image = np.expand_dims(image, axis=0)

In [ ]:
pred_mask = model.predict(image)
pred_mask = (pred_mask > 0.5).astype('int32')
plt.figure(figsize=(15,8))
plt.subplot(131)
plt.title('Original Image')
plt.imshow(org_image)
plt.axis('off')
plt.subplot(132)
plt.title('Processed Image')
plt.imshow(image[0], cmap='gray')
plt.axis('off')
plt.subplot(133)
plt.title('Predicted Mask')
plt.imshow(pred_mask[0], cmap='gray')
plt.axis('off')

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2

def save_predicted_mask(org_image_path, model, output_path, IMAGE_SIZE=512):
    # Load the original image
    org_image = plt.imread(org_image_path)
    
    # Preprocess the image
    image = org_image[:, :, 1]
    processed_img = preprocess(image)
    processed_img = (processed_img * 255).astype(np.uint8)
    image = tf.convert_to_tensor(processed_img)
    image.set_shape([None, None, 3])
    image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
    image = image / 255.
    image = np.expand_dims(image, axis=0)

    # Predict the mask
    pred_mask = model.predict(image)
    pred_mask = (pred_mask > 0.5).astype('int32')
    pred_mask = pred_mask[0][:, :, 0]

    # Create a subplot to display the original and predicted images
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    # Display the original image
    axes[0].imshow(org_image)
    axes[0].set_title('Original Image')

    # Display the predicted mask
    axes[1].imshow(pred_mask, cmap='gray')
    axes[1].set_title('Predicted Mask')

    # Save the predicted mask with a suffix
    save_dir = os.path.dirname(output_path)
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist
    base_name = os.path.splitext(os.path.basename(output_path))[0]

    pred_save_path = os.path.join(save_dir, f'{base_name}.png')
    plt.imsave(pred_save_path, pred_mask, cmap='gray')

    # Close the figure to free up resources
    plt.close(fig)

# Example usage:
for i in range(1, 21): 
    org_image_path = f'/kaggle/input/neu-2dsegmentation/segmentation/test/image/{i}.jpg'
    save_image_path = f'/kaggle/working/answer3/{i}.png'
    save_predicted_mask(org_image_path, model, save_image_path)


In [ ]:
import shutil

def create_zip(source_folder, output_path):
    shutil.make_archive(output_path, 'zip', source_folder)

# 指定目标文件夹和压缩文件路径
source_directory = '/kaggle/working/answer3'
zip_file_path = '/kaggle/working/answer3'

# 调用函数进行压缩
create_zip(source_directory, zip_file_path)



# Our Low Quality Image Dataset

In [ ]:
IMAGE_SIZE = 512

org_image = plt.imread('../input/our-retinal-fundus-image-dataset/HHFC Images/HHFC Images/EL 180624436112213.jpg')
image = org_image[:,:,1]
processed_img = preprocess(image)
# processed_img = Image.fromarray((processed_img * 255).astype(np.uint8))
processed_img = (processed_img * 255).astype(np.uint8)
image = tf.convert_to_tensor(processed_img)
image.set_shape([None, None, 3])
image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
image = image / 255.
image = np.expand_dims(image, axis=0)

In [ ]:
pred_mask = model.predict(image)
pred_mask = (pred_mask > 0.5).astype('int32')
plt.figure(figsize=(15,8))
plt.subplot(131)
plt.title('Original Image')
plt.imshow(org_image)
plt.axis('off')
plt.subplot(132)
plt.title('Processed Image')
plt.imshow(image[0], cmap='gray')
plt.axis('off')
plt.subplot(133)
plt.title('Predicted Mask')
plt.imshow(pred_mask[0], cmap='gray')
plt.axis('off')

<a href='./unet_backbone_vgg19.h5'>Download VGG19 Model</a>